## BERTopic

In [2]:
import numpy as np
from bertopic import BERTopic
from gensim import corpora
from sentence_transformers import SentenceTransformer
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import CoherenceModel
from hdbscan import HDBSCAN
from umap import UMAP
import gensim.corpora as corpora
import pandas as pd
import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd
import numpy as np
import os

import nltk
from nltk.corpus import PlaintextCorpusReader

c:\Users\leele\anaconda3\envs\TAA2023\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import preprocess2
reviews_corpus = preprocess2.load_corpus('sephora_corpus')
reviews_docs = preprocess2.corpus2docs(reviews_corpus)


# # Build the bigram and trigram models
# bigram = gensim.models.Phrases(reviews_docs, min_count=5, threshold=100) # higher threshold fewer phrases. 

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)

# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]

# data_words_bigrams = make_bigrams(reviews_docs)

reviews_docs_joined = [" ".join(x) for x in reviews_docs]

In [4]:
from keybert import KeyBERT
# Extract keywords
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(reviews_docs_joined )

# Create our vocabulary
vocabulary = [k[0] for keyword in keywords for k in keyword]
vocabulary = list(set(vocabulary))

In [5]:
from bertopic import BERTopic
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from sklearn.cluster import KMeans

In [6]:
vectorizer_model= CountVectorizer(vocabulary=vocabulary)
embedding_model = SentenceTransformer('all-mpnet-base-v2')
embeddings = embedding_model.encode(reviews_docs_joined, show_progress_bar=False)
representation_model = KeyBERTInspired() #to reduce the appearance of stop words
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True) #to reduce the impact of frequent words

In [8]:

 # Dimensionality Reduction Parameters
n_neighbors_range = [5, 10, 15]
n_components_range = [3, 5, 7]

# Define the range of parameters for clustering
hdbscan_model = HDBSCAN()
kmeans_model = KMeans(n_clusters=7)

cluster_range = [hdbscan_model, kmeans_model]

# Define the range of parameters for ngrams
unigram = (1, 1)
bigram = (1, 2)
ngram_range = [unigram, bigram]

model_results = {'N_Neighbors': [],
                 'N_Components': [],
                 'Clustering_methods': [],
                 'Ngram_range': [],
                 'Coherence': []
                }

# Define the size of the subset (10%)
subset_size = int(len(reviews_docs_joined) * 0.3)
subset_docs = np.random.choice(reviews_docs_joined, subset_size, replace=False)

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(n_neighbors_range) *len(n_components_range) * len(cluster_range) * len(ngram_range)))

for n_neighbors in n_neighbors_range:
    for n_components in n_components_range:
        for cluster in cluster_range:
            for ngram in ngram_range:
                umap_model = UMAP(n_neighbors=n_neighbors, n_components=n_components, low_memory=True, min_dist=0.0, metric="cosine")
                # Create the BERTopic model
                topic_model = BERTopic(embedding_model=embedding_model, umap_model= umap_model, hdbscan_model=cluster, n_gram_range = ngram, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model, representation_model=representation_model)
                
                print(f'N_Neighbors: {n_neighbors}')
                print(f'N_Components: {n_components}')
                print(f'Clustering_methods: {cluster}')
                print(f'Ngram_range: {ngram}')

            # Fit the model on your subset of documents
                topics, _ = topic_model.fit_transform(subset_docs)
                
                # Preprocess documents
                documents = pd.DataFrame(
                    {"Document": subset_docs,
                    "ID": range(len(subset_docs)),
                    "Topic": topics}
                )
                documents_per_topic = documents.groupby(
                    ['Topic'], as_index=False).agg({'Document': ' '.join})
                cleaned_docs = topic_model._preprocess_text(
                    documents_per_topic.Document.values)
                
                # Extract vectorizer and analyzer from the fitted model
                vectorizer_model = topic_model.vectorizer_model
                analyzer = vectorizer_model.build_analyzer()
                
                # Extract features for topic coherence evaluation
                tokens = [analyzer(doc) for doc in cleaned_docs]
                dictionary = corpora.Dictionary(tokens)
                corpus = [dictionary.doc2bow(token) for token in tokens]
                topic_words = [[words for words, _ in topic_model.get_topic(topic)] for topic in range(len(set(topics)) - 1)]
                
                # Calculate coherence
                coherence_model = CoherenceModel(topics=topic_words, texts=tokens, corpus=corpus, dictionary=dictionary, coherence='c_v')
                coherence_score = coherence_model.get_coherence()
                print(f'Coherence score: {coherence_score}')

                # Get list of topics
                print(topic_model.get_topic_info())
                
                # Save the model results
                model_results['N_Neighbors'].append(n_neighbors)
                model_results['N_Components'].append(n_components)
                model_results['Clustering_methods'].append(cluster)
                model_results['Ngram_range'].append(ngram)
                model_results['Coherence'].append(coherence_score)

                pbar.update(1)
    pbar.close()

  0%|          | 0/36 [14:10<?, ?it/s]


N_Neighbors: 5
N_Components: 3
Clustering_methods: HDBSCAN()
Ngram_range: (1, 1)


c:\Users\leele\anaconda3\envs\TAA2023\lib\site-packages\bertopic\vectorizers\_ctfidf.py:69: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df)+1)


Coherence score: 0.3754327050489134
     Topic  Count                                              Name  \
0       -1  10484      -1_moisturizes_concealer_regimen_niacinamide   
1        0    951            0_lipstick_lipglosses_lipgloss_tasting   
2        1    712                 1_mask_masks_maskne_moisturizermy   
3        2    394                      2_tanning_tan_tanned_tanluxe   
4        3    267     3_mascara_makeupespecially_makeupnot_eyeliner   
..     ...    ...                                               ...   
793    792      5     792_moisturizing_moisturizer_skin_brightening   
794    793      5       793_cleanser_exfoliator_moisturizer_cleanse   
795    794      5                     794_skincare_serum_skin_toner   
796    795      5  795_moisturize_moisturizer_moisturizing_skincare   
797    796      5     796_exfoliant_exfoliants_exfoliating_cleanser   

                                        Representation  \
0    [moisturizes, concealer, regimen, niacinamide,..

c:\Users\leele\anaconda3\envs\TAA2023\lib\site-packages\bertopic\vectorizers\_ctfidf.py:69: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df)+1)


...

Coherence score: 0.37634645331523464
     Topic  Count                                               Name  \
0       -1  13141        -1_moisturizers_concealer_lotion_hyaluronic   
1        0   1926       0_cleanser_exfoliator_exfoliators_exfoliants   
2        1    945                 1_lipstick_lipglosses_lipgloss_lip   
3        2    693                    2_exfoliation_mask_masks_maskne   
4        3    643              3_sunscreen_suncreen_sunblock_sunburn   
..     ...    ...                                                ...   
396    395      5                 395_moisturizer_serum_makeup_cream   
397    396      5                   396_skin_oily_greasy_niacinamide   
398    397      5                   397_scent_smell_moisture_product   
399    398      5  398_moisturizer_moisturizers_moisturizing_frag...   
400    399      5                 399_moisturizing_face_skin_massage   

                                        Representation  \
0    [moisturizers, concealer, lotion, h

c:\Users\leele\anaconda3\envs\TAA2023\lib\site-packages\bertopic\vectorizers\_ctfidf.py:69: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df)+1)


Coherence score: 0.5512787072027406
   Topic  Count                                              Name  \
0      0  10107                 0_skincare_serum_moisturizer_acne   
1      1   7849  1_skincare_moisturizer_moisturizers_moisturizing   
2      2   2959       2_exfoliator_exfoliant_cleanser_exfoliating   
3      3   2762             3_sunscreen_lotion_moisturizer_makeup   
4      4   2212               4_moisturizing_concealer_serum_balm   
5      5    949                      5_lipstick_lipgloss_lip_lips   
6      6     89                         6_bonne_vraiment_pour_bon   

                                      Representation  \
0  [skincare, serum, moisturizer, acne, retinol, ...   
1  [skincare, moisturizer, moisturizers, moisturi...   
2  [exfoliator, exfoliant, cleanser, exfoliating,...   
3  [sunscreen, lotion, moisturizer, makeup, tanni...   
4  [moisturizing, concealer, serum, balm, cream, ...   
5  [lipstick, lipgloss, lip, lips, balm, gloss, v...   
6  [bonne, vraiment

c:\Users\leele\anaconda3\envs\TAA2023\lib\site-packages\bertopic\vectorizers\_ctfidf.py:69: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df)+1)


Coherence score: 0.48581239718216285
   Topic  Count                                              Name  \
0      0   9286  0_moisturizer_skincare_moisturizers_moisturizing   
1      1   7142              1_serum_moisturizer_skincare_retinol   
2      2   4318                     2_skincare_acne_rosacea_serum   
3      3   2849             3_sunscreen_makeup_lotion_moisturizer   
4      4   2185               4_concealer_moisturizing_balm_serum   
5      5   1058             5_lipstick_balm_lipgloss_moisturizing   
6      6     89                         6_bonne_vraiment_pour_bon   

                                      Representation  \
0  [moisturizer, skincare, moisturizers, moisturi...   
1  [serum, moisturizer, skincare, retinol, produc...   
2  [skincare, acne, rosacea, serum, retinol, skin...   
3  [sunscreen, makeup, lotion, moisturizer, tanni...   
4  [concealer, moisturizing, balm, serum, cream, ...   
5  [lipstick, balm, lipgloss, moisturizing, lip, ...   
6  [bonne, vraimen

In [9]:
pd.DataFrame(model_results).to_csv('./bertopic_tuning_results.csv', index=False)

In [10]:
map_model = UMAP(n_neighbors=15, n_components=7, low_memory=True, min_dist=0.0, metric="cosine")
topic_model = BERTopic(embedding_model=embedding_model, umap_model= umap_model, hdbscan_model=kmeans_model, n_gram_range = (1,1), vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model, representation_model=representation_model)
topics = topic_model.fit_transform(reviews_docs_joined)

print(topic_model.get_topic_info())

   Topic  Count                                              Name  \
0      0  27359  0_moisturizer_skincare_moisturizers_moisturizing   
1      1  18966               1_acne_skincare_retinol_moisturizer   
2      2  14114                 2_serum_moisturizer_cream_product   
3      3   9890     3_cleanser_exfoliator_exfoliating_exfoliation   
4      4   8518                4_sunscreen_moisturizer_lotion_spf   
5      5   7706               5_moisturizing_serum_concealer_balm   
6      6   3205             6_lipstick_lipgloss_moisturizing_balm   

                                      Representation  \
0  [moisturizer, skincare, moisturizers, moisturi...   
1  [acne, skincare, retinol, moisturizer, serum, ...   
2  [serum, moisturizer, cream, product, fragrance...   
3  [cleanser, exfoliator, exfoliating, exfoliatio...   
4  [sunscreen, moisturizer, lotion, spf, makeup, ...   
5  [moisturizing, serum, concealer, balm, cream, ...   
6  [lipstick, lipgloss, moisturizing, balm, lip, ...   

In [11]:
map_model = UMAP(n_neighbors=15, n_components=7, low_memory=True, min_dist=0.0, metric="cosine")
topic_model = BERTopic(embedding_model=embedding_model, umap_model= umap_model, hdbscan_model=hdbscan_model, n_gram_range = (1,1), vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model, representation_model=representation_model)
topics = topic_model.fit_transform(reviews_docs_joined)

print(topic_model.get_topic_info())

      Topic  Count                                             Name  \
0        -1  50485    -1_moisturizes_acneprone_concealer_complexion   
1         0   3174            0_lipstick_lipgloss_lipbalm_chapstick   
2         1   2239                  1_masks_mask_maskne_exfoliation   
3         2   1112  2_leavesskin_dermalogical_innbeautys_sandalwood   
4         3   1111                     3_tanning_tan_tanned_bronzer   
...     ...    ...                                              ...   
1107   1106      5                  1106_shade_shades_blended_blend   
1108   1107      5       1107_complexion_vitamin_application_months   
1109   1108      5     1108_moisturizing_serumlike_applicator_cream   
1110   1109      5               1109_fragrance_scent_product_smell   
1111   1110      5    1110_moisturizer_moisturizers_spatula_product   

                                         Representation  \
0     [moisturizes, acneprone, concealer, complexion...   
1     [lipstick, lipgloss, li